In [25]:
import pandas as pd
# Load member information
all_accounts_df = pd.read_csv('~/Documents/pyoverlee/ms_exports/accounts-2024-09-26.csv')
# Load visit data

visits_df = pd.read_csv('~/Documents/pyoverlee/ms_exports/2024-visits-2024-09-20.csv')
# Replace spaces with underscores and convert to lower case
all_accounts_df.columns = all_accounts_df.columns.str.replace(' ', '_', regex=False).str.lower()
# Standardize 'acct_type' and 'payment_status' values
all_accounts_df['acct_type'] = all_accounts_df['acct_type'].str.lower().str.strip()
all_accounts_df['payment_status'] = all_accounts_df['payment_status'].str.lower().str.strip()
all_accounts_df.rename(columns={'acct_#': 'acct_id'}, inplace=True)
# Replace spaces with underscores and convert to lower case
visits_df.columns = visits_df.columns.str.replace(' ', '_', regex=False).str.lower()
visits_df.head()


,checkin_time,acct_id,account_type,name,member_type,type,num_credits
0,2024-09-20 08:45:24,10318,Active Summer,Bill Berlin,Adult,member,0
1,2024-09-20 08:39:09,11009,Active Summer,Holly Yoshinari,Adult,member,0
2,2024-09-20 08:38:18,10077,Active Summer,Mary Bohan,Adult,member,0
3,2024-09-20 08:19:08,10396,Active Summer,Monique Hanis,Adult,member,0
4,2024-09-20 08:11:37,10206,Active Summer,Mark Cummings,Adult,member,0


In [26]:
num_rows = len(all_accounts_df)
print(num_rows, " Total Number of Accounts in Membersplash")
num_rows = len(visits_df)
print(num_rows, " Total Number of Visits in Membersplash")

2540  Total Number of Accounts in Membersplash
44425  Total Number of Visits in Membersplash


In [27]:
# Get unique account types
unique_account_types = all_accounts_df['acct_type'].unique()
print("Unique account types:")
print(unique_account_types)
unique_account_types_vists = visits_df['account_type'].unique()
print("Unique account types in Visits:")
print(unique_account_types_vists)

Unique account types:
['active summer' 'gold pass' 'inactive' 'waitlist' 'offered membership'
 'august pass (waitlist)' 'test admin' 'august pass (non member)'
 'non-members (august pass)' 'august pass (member)']
Unique account types in Visits:
['Active Summer' 'Gold Pass' 'August Pass (Waitlist)'
 'August Pass (Non Member)' 'Offered Membership' 'August Pass (Member)'
 'Waitlist' 'Honorary Pass']


In [28]:
# Calculate the counts of each account type
account_type_counts = all_accounts_df['acct_type'].value_counts()
print("Total number of members for each account type:")
print(account_type_counts)

Total number of members for each account type:
acct_type
waitlist                     1319
active summer                 851
gold pass                     168
august pass (waitlist)         82
inactive                       64
august pass (non member)       40
non-members (august pass)      11
test admin                      3
offered membership              1
august pass (member)            1
Name: count, dtype: int64


## Account Type Definitions (Large update each Fall and February - whenever people pay)

- **Active Summer**: Eligible for Summer Membership - Includes Unpaid accounts
- **Gold Pass**: Limited Use Membership
- **Waitlist**: Paid to be on the Waitlist (Includes people who declined or didn't respond for the second time in 2024)
- **Offered Membership**: 
- **Honorary Pass**: 
- **Inactive**: 
- **Test Admin**: 

### August Pass Types

- **August Pass (Waitlist)**: People on the Waitlist who have purchased an August Pass
- **August Pass (Non Member)**: People who purchased an August Pass who are neither on the Waitlist nor a Member
- **August Pass (Member)**: People who are Members who have purchased an August Pass
- **Non-Members (August Pass)**: People who have previously purchased an August Pass

In [29]:
# Define the target account types
august_target_account_types = ['August Pass (Waitlist)', 'August Pass (Member)','August Pass (Non Member)']
# Filter members based on the target account types
filtered_members_df = all_accounts_df[all_accounts_df['acct_type'].isin(august_target_account_types)]
# Identify unique account #s
all_members = set(filtered_members_df['acct_id'].unique())
print(len(all_members), " Total August Pass Accounts")

#TODO: Filter out 43 people Unpaid 


0  Total August Pass Accounts


In [30]:
# Get unique member IDs who visited
visiting_member_ids = visits_df['acct_id'].unique()
# Create a DataFrame indicating whether each member_id has visited
visited_df = pd.DataFrame({'acct_id': visiting_member_ids})
visited_df['visited'] = True

# Merge members_df with visited_df
members_visited_df = all_accounts_df.merge(visited_df, on='acct_id', how='left')

# Fill NaN values in 'visited' column with False (members who did not visit)
members_visited_df['visited'] = members_visited_df['visited'].fillna(False)

In [32]:
# Create a crosstab of 'acct_type' and 'visited'
#visit_crosstab = pd.crosstab(members_visited_df['acct_type'], members_visited_df['visited'])

# Create a crosstab of 'acct_type', 'payment_status', and 'visited'
visit_crosstab = pd.crosstab(
    [members_visited_df['acct_type'], members_visited_df['payment_status']],  # Rows: acct_type and payment_status
    members_visited_df['visited']  # Columns: visited status
)

# Reset index to turn 'acct_type' and 'payment_status' back into columns
visit_crosstab = visit_crosstab.reset_index()

# Rename columns for clarity
visit_crosstab = visit_crosstab.rename(columns={False: 'did_not_visit', True: 'visited'})

# Calculate total members per account type
visit_crosstab['total_accounts'] = visit_crosstab['visited'] + visit_crosstab['did_not_visit']

# Reorder columns for better readability
#visit_crosstab = visit_crosstab[['acct_type', 'total_accounts', 'visited', 'did_not_visit']]
# Reorder columns
visit_crosstab = visit_crosstab[['acct_type', 'payment_status', 'total_accounts', 'visited', 'did_not_visit']]


# Display the result
print(visit_crosstab)


visited                  acct_type payment_status  total_accounts  visited  \
0                    active summer           paid             808      783   
1                    active summer         unpaid              43        3   
2             august pass (member)           paid               1        1   
3         august pass (non member)           paid              40       39   
4           august pass (waitlist)           paid              82       79   
5                        gold pass           paid             168      119   
6                         inactive           paid              64        0   
7        non-members (august pass)           paid              11        0   
8               offered membership         unpaid               1        1   
9                       test admin           paid               2        0   
10                      test admin         unpaid               1        0   
11                        waitlist           paid            131

In [34]:
# Define the target account type
target_account_type = 'active summer'  # Replace with the actual account type from your data

# Filter members who belong to the target account type and did not visit
unpaid_visiting_members = members_visited_df[
    (members_visited_df['acct_type'] == target_account_type) &
    (members_visited_df['visited'] == True) &
    (members_visited_df['payment_status'] == 'unpaid')
]

# Display the non-visiting member(s)
print(f"\Visiting member(s) in account type '{target_account_type}':")
print(unpaid_visiting_members)

\Visiting member(s) in account type 'active summer':
     acct_id      acct_type                tags date_registered  \
352    10380  active summer                 NaN      2011-05-01   
382    10416  active summer  Gold Pass Eligible      2005-05-01   
555    10599  active summer  Gold Pass Eligible      2000-05-01   

     legacy_acct_#                      user_name first_name  last_name  \
352         3017.0  josh@joshgreenberglawfirm.com       Josh  Greenberg   
382         2563.0          kohls_heavner@msn.com      Brett    Heavner   
555         2260.0          stephen.marino@me.com    Stephen     Marino   

                             email  site_access  ...    zip    cell_phone  \
352  josh@joshgreenberglawfirm.com            1  ...  22207           NaN   
382          kohls_heavner@msn.com            1  ...  22207  703-869-7301   
555          stephen.marino@me.com            1  ...  22205  202-841-2941   

    work_phone    home_phone payment_status checkin_note #_members  

In [21]:
# Define the target account type
target_account_type = 'Waitlist'  # Replace with the actual account type from your data

# Filter members who belong to the target account type and did not visit
waitlist_visiting_members = members_visited_df[
    (members_visited_df['acct_type'] == target_account_type) &
    (members_visited_df['visited'] == True)
]

# Display the non-visiting member(s)
print(f"\Visiting member(s) in account type '{target_account_type}':")
print(waitlist_visiting_members)


\Visiting member(s) in account type 'Waitlist':
      acct_id acct_type tags date_registered  legacy_acct_#    user_name  \
2263    12412  Waitlist  NaN      2023-04-29            NaN  hooperaj621   

     first_name last_name                  email  site_access  ...    zip  \
2263      Aimee    Hooper  hooperaj621@gmail.com            1  ...  22205   

      cell_phone work_phone    home_phone payment_status checkin_note  \
2263         NaN        NaN  518-322-2020           paid          NaN   

     #_members authnet_account                  member_names visited  
2263         2             NaN  Aimee Hooper, Enzio Quartero    True  

[1 rows x 27 columns]


In [37]:
waitlist_visits = visits_df[
    (visits_df['acct_id'] == 10599)
]
print(waitlist_visits)
# August Pass (Waitlist) moved back

              checkin_time  acct_id   account_type           name member_type  \
19156  2024-07-16 14:16:39    10599  Active Summer  Alison Marino       Adult   

         type  num_credits  
19156  member            0  
